# Incorporating micro-feedback effects

In [1]:
import sys
sys.path.append("../../")
import taxcalc
from taxcalc import *
import pandas as pd
import numpy as np

### Start by creating a Calculator for Plan X and a calculator for Plan Y. 

In [2]:
# Create a Records object for Plan X and Plan Y
records_x = Records("../../puf.csv")
records_y = Records("../../puf.csv")
# Create a Parameters object for Plan X and Plan Y
params_x = Parameters()
params_y = Parameters()
behavior_y = Behavior()

# Create two Calculators
calcX = Calculator(params_x, records_x)
calcY = Calculator(params_y, records_y, behavior=behavior_y)


You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


/Users/Matt.Jensen/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (204,205) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


### Increase the top marginal tax rate by 10 percentage points 

In [3]:
# Modify the relevant parameter for the Plan-Y Calculator.
reform = {2013: {"_II_rt7": [0.496]}}
params_y.implement_reform(reform)
# Demonstrate that Plan X and Plan Y calculators are indeed different. 
print(calcX.params.II_rt7)
print(calcY.params.II_rt7)

0.396
0.496


### Update taxpayers' income to account for the rate hike with our behavioral effects calculator

In [4]:
# Call the behavioral effects calculator and create new Plan Y Calculator obects. 
behavior_y.update_behavior({2013: {'_BE_inc': [0.15], '_BE_sub': [0.4]}})
calcY_behavior1 = behavior(calcX, calcY)
behavior_y.update_behavior({2013: {'_BE_inc': [0.15], '_BE_sub': [0.5]}})
calcY_behavior2 = behavior(calcX, calcY)
behavior_y.update_behavior({2013: {'_BE_inc': [0.0], '_BE_sub': [0.4]}})
calcY_behavior3 = behavior(calcX, calcY)                           

In [5]:
# Demonstrate that taxpayers' income was affected by the behavioral effects.
print(calcY.records.e00200.sum())
print(calcY_behavior1.records.e00200.sum())
print(calcY_behavior2.records.e00200.sum())
print(calcY_behavior3.records.e00200.sum())

32329002724
31047019924.0
30671634834.6
30827462366.3


In [6]:
# Demonstrate that total tax was affected by the behavioral effects. 
print(calcX.records._ospctax.sum())
print(calcY.records._ospctax.sum())
print(calcY_behavior1.records._ospctax.sum())
print(calcY_behavior2.records._ospctax.sum())
print(calcY_behavior3.records._ospctax.sum())

31480202328.8
36096622590.6
33957881240.1
33349777977.8
33609522097.7
